In [ ]:
from utils.utils import build_market_image, prepare_forecast_data
from neuralforecast.core import NeuralForecast
from ray.rllib.policy.policy import Policy
# from environments.live_environments import LiveTradingEnv
from utils.clients import AlpacaClient,OandaClient,CoinbaseClient
from Keys import *
from utils.mappings import oanda_time_map
from utils.reward_functions import sharpe_reward_function
import pandas as pd

In [2]:
forecasting_model=NeuralForecast.load('MultiHeadForecastingModel/')
agent_dir='Agent/final_checkpoints/policies/default_policy'
agent= Policy.from_checkpoint(agent_dir)

Seed set to 1
Seed set to 15
Seed set to 16


In [3]:
base_asset='ETH'
quote_asset='USD'
test_net=True
time_frame='1h'
trade_target='/'.join([base_asset,quote_asset])
positions = [-1,0,1]

In [ ]:
client=CoinbaseClient(api_key=alpaca_api_key,
                    api_secret=alpaca_api_secret,
                
                    paper=test_net,
                    symbol=trade_target,
                    time_frame=time_frame)


[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('a9f22f43-80c5-42e9-82a4-2c83eecbcf7d'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Emmis Corporation Common Stock',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.INACTIVE: 'inactive'>,
     'symbol': 'EMMS',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.BATS: 'BATS'>,
     'fractionable': False,
     'id': UUID('509eb6ef-e22e-41fe-acc7-475128889ac6'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': '',
     'price_increment': None,


In [10]:
asset_list=client.trade_client.get_all_assets()

In [ ]:
asset=asset_list[0]


{'id': UUID('a9f22f43-80c5-42e9-82a4-2c83eecbcf7d'),
 'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
 'exchange': <AssetExchange.OTC: 'OTC'>,
 'symbol': 'EMMS',
 'name': 'Emmis Corporation Common Stock',
 'status': <AssetStatus.INACTIVE: 'inactive'>,
 'tradable': False,
 'marginable': False,
 'shortable': False,
 'easy_to_borrow': False,
 'fractionable': False,
 'min_order_size': None,
 'min_trade_increment': None,
 'price_increment': None,
 'maintenance_margin_requirement': 100.0,
 'attributes': []}

In [48]:
asset_frame=pd.DataFrame.from_records([asset.__dict__ for asset in asset_list ])

In [ ]:

asset_frame['asset_class']=asset_frame['asset_class'].apply(lambda x: str(x))
asset_frame['asset_class']=asset_frame['asset_class'].str.replace('AssetClass.','')
asset_frame['exchange']=asset_frame['exchange'].apply(lambda x: str(x))
asset_frame['exchange']=asset_frame['exchange'].str.replace('AssetExchange.','')
asset_frame['status']=asset_frame['status'].apply(lambda x: str(x))
asset_frame['status']=asset_frame['status'].str.replace('AssetStatus.','')
asset_frame=asset_frame[asset_frame['tradable']==True].reset_index(drop=True)
shortable=asset_frame[asset_frame['marginable']==True].reset_index(drop=True)

shortable['asset_class'].value_counts()
# shortable

asset_class
US_EQUITY    11033
Name: count, dtype: int64

In [ ]:
client.get_balance(symbol=quote_asset)

In [ ]:
live_env_config=dict(
                name=f'{trade_target}_live',
                model=model,
                agent=agent,
                api_key=oanda_api_key,
                account_id=oanda_account_id,
                test_net=True,
                restore_trading=True,
                target_symbol=trade_target,
                time_frame=time_frame,
                reward_function=sharpe_reward_function,
                positions = positions, # -1 (=SHORT), 0(=OUT), +1 (=LONG)
                trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
                borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
                max_episode_duration=168,
                verbose=0,
                )

live_env=OandaTradingEnv(**live_env_config)

In [ ]:
trade_target